<a href="https://colab.research.google.com/github/baudouindetruchis/VRP-RL/blob/main/VRP_DQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Ressources**
1. [AlphaZero paper](https://www.nature.com/articles/nature24270.epdf?author_access_token=VJXbVjaSHxFoctQQ4p2k4tRgN0jAjWel9jnR3ZoTv0PVW4gB86EEpGqTRDtpIz-2rmo8-KG06gqVobU5NSCFeHILHcVFUeMsbvwS-lxjqQGg98faovwjxeTUgZAUMnRQ)
2. [How to build AlphaZero algo](https://medium.com/applied-data-science/how-to-build-your-own-alphazero-ai-using-python-and-keras-7f664945c188)
3. [Colab tips](https://towardsdatascience.com/10-tips-for-a-better-google-colab-experience-33f8fe721b82)

**Ideas**
1. Optimize types (float < **int**)
2. Transfer learning for feature extraction --> speed-up learning



In [ ]:
import numpy as np
import random
import os
import time
from collections import deque
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Activation, ZeroPadding2D, Add, Input
from keras.utils.vis_utils import plot_model
from tqdm.notebook import tqdm
import tensorflow as tf
from keras.callbacks import TensorBoard


# Load tensorboard notebook extension (with magic command %)
# %load_ext tensorboard

# Check GPU
if not len(tf.test.gpu_device_name()):
    print("[INFO] GPU not activated")

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## Logging

In [ ]:
# Own Tensorboard class
class ModifiedTensorBoard(TensorBoard):

    # Overriding init to set initial step and writer (we want one log file for all .fit() calls)
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.step = 1
        # self.writer = tf.summary.FileWriter(self.log_dir)
        self.writer = tf.summary.create_file_writer(self.log_dir)

    # Overriding this method to stop creating default log writer
    def set_model(self, model):
        pass

    # Overrided, saves logs with our step number
    # (otherwise every .fit() will start writing from 0th step)
    def on_epoch_end(self, epoch, logs=None):
        self.update_stats(**logs)

    # Overrided
    # We train for one batch only, no need to save anything at epoch end
    def on_batch_end(self, batch, logs=None):
        pass

    # Overrided, so won't close writer
    def on_train_end(self, _):
        pass

    # Custom method for saving own metrics
    # Creates writer, writes custom metrics and closes writer
    def update_stats(self, **stats):
        self._write_logs(stats, self.step)

## Parameters

In [ ]:
##### ENVIRONMENT
GRID_SIZE = 10
N_POD = 15

##### SELF PLAY
EPISODES = 20_000
DISCOUNT = 0.99
EPSILON = 1     # variable which is going to be decayed
EPSILON_DECAY = 0.99975
MIN_EPSILON = 0.001

##### MODEL
LOAD_MODEL = None  # Filepath or None
MODEL_NAME = '3x256C'
HISTORY_SIZE = 2

##### RETRAINING
REPLAY_MEMORY_SIZE = 50_000
MIN_REPLAY_MEMORY_SIZE = 1_000  # Minimum number of steps in a memory to start training
MINIBATCH_SIZE = 64  # How many steps (samples) to use for training
UPDATE_TARGET_EVERY = 5  # Terminal states (end of episodes)

##### STATS MONITORING
MIN_REWARD = -65   # For model save
AGGREGATE_STATS_EVERY = 50      # episodes

## Environment

In [ ]:
class VRP:
    def __init__(self, grid_size=GRID_SIZE, n_pod=N_POD, history_size=HISTORY_SIZE):
        # Parameters
        self.grid_size = grid_size
        self.n_pod = n_pod
        self.history_size = history_size

        # Game states
        self.depot_grid = self.create_depot_grid()
        self.pod_grid = self.create_pod_grid()
        self.history = np.concatenate((np.expand_dims(self.depot_grid, axis=2), np.zeros((self.grid_size, self.grid_size, history_size), dtype=int)), axis=2)
        
        # Variables
        self.episode_step = 0
        self.route = [np.unravel_index(np.argmax(self.depot_grid, axis=None), self.depot_grid.shape)]     # Stores vehicule route

    # Give the position of the depot
    def create_depot_grid(self):
        depot_grid = np.zeros((self.grid_size, self.grid_size), dtype=int)
        depot_grid[np.random.randint(0, self.grid_size - 1), random.randint(0, self.grid_size - 1)] = 1
        return depot_grid

    # Give position of PoD
    def create_pod_grid(self):
        pod_grid = np.zeros((self.grid_size, self.grid_size), dtype=int)
        for i in range(self.n_pod):
            random_x = np.random.randint(0, self.grid_size - 1)
            random_y = np.random.randint(0, self.grid_size - 1)
            # Reset if spot already taken
            while (self.depot_grid[random_x, random_y] == 1) or (pod_grid[random_x, random_y] == 1):
                random_x = np.random.randint(0, self.grid_size - 1)
                random_y = np.random.randint(0, self.grid_size - 1)
            pod_grid[random_x, random_y] = 1
        return pod_grid
    
    def get_observation(self):
        # Stack (create a new dimension) depot and PoD grid
        observation = np.concatenate((np.expand_dims(self.depot_grid, axis=2), np.expand_dims(self.pod_grid, axis=2)), axis=2)
        observation = np.concatenate((observation, self.history), axis=2)
        return observation
    
    # Return coordinates of the max valid action
    def max_valid_action(self, prediction):
        # Valid actions: not yet visited PoD
        valid_actions = (self.depot_grid + self.pod_grid - self.history[:,:,0]).astype(bool)
        prediction[~valid_actions] = -np.inf    # NN can output negative values
        max_valid_coordinates = np.unravel_index(np.argmax(prediction, axis=None), prediction.shape)    # Argmax in n-dimension array
        return max_valid_coordinates
    
    # Distance as the crow flies
    def get_total_distance(self):
        distance = 0
        for i in range(len(env.route)-1):
            distance += np.sqrt((env.route[i+1][0]-env.route[i][0])**2 + (env.route[i+1][1]-env.route[i][1])**2)
        return round(distance, 1)
    
    # Last step distance
    def get_step_distance(self):
        distance = np.sqrt((env.route[-1][0]-env.route[-2][0])**2 + (env.route[-1][1]-env.route[-2][1])**2)
        return round(distance, 1)
    
    def step(self, action):
        self.episode_step += 1
        # Update game states
        new_state = self.history[:,:,0].copy()
        new_state[action[0], action[1]] = 1
        self.history = np.concatenate((np.expand_dims(new_state, axis=2), self.history[:,:,:-1]), axis=2).copy()
        # Save position for distance calculation
        current_position = self.history[:,:,0] - self.history[:,:,1]
        self.route.append(np.unravel_index(np.argmax(current_position, axis=None), current_position.shape))
        # Check if episode finished
        if (self.depot_grid + self.pod_grid == self.history[:,:,0]).all():
            done = True
            self.route.append(np.unravel_index(np.argmax(self.depot_grid, axis=None), self.depot_grid.shape))
            reward = - (self.get_step_distance() + self.get_total_distance())
        else:
            done = False
            reward = -self.get_step_distance()
        return self.get_observation(), reward, done

env = VRP()

## Agent

In [ ]:
class Agent:
    def __init__(self):
        # Main model (train every step)
        self.model = self.create_model()
        # Target network (.predict every step --> updated to main model after X complete episode)
        self.target_model = self.create_model()
        self.target_model.set_weights(self.model.get_weights())

        # An array with last n steps for training (create a batch for updates)
        self.replay_memory = deque(maxlen=REPLAY_MEMORY_SIZE)
        # Used to count when to update target network with main network's weights
        self.target_update_counter = 0
        # Custom tensorboard object
        # self.tensorboard = ModifiedTensorBoard(log_dir=f"logs/{MODEL_NAME}_{int(time.time())}")
    
    def create_model(self):
        if LOAD_MODEL:
            print(f"[INFO] Loading model: {LOAD_MODEL}")
            model = load_model(LOAD_MODEL)
            print("[INFO] Model loaded")
        else:
            X_input = Input(shape=(env.grid_size, env.grid_size, 3 + env.history_size))

            X = ZeroPadding2D((2, 2))(X_input)      # Zero-Padding to keep borders info

            X_shortcut = X      # Save the input value

            X = Conv2D(256, (3, 3), padding = 'same', activation = 'relu')(X)
            X = BatchNormalization()(X)  # Applied after non linearity (other say before)
            X = Dropout(0.1)(X)

            X = Conv2D(256, (3, 3), padding = 'same', activation = 'relu')(X)
            X = BatchNormalization()(X)
            X = Dropout(0.1)(X)

            X = Conv2D(256, (3, 3), padding = 'same', activation = 'relu')(X)
            X = BatchNormalization()(X)
            X = Dropout(0.1)(X)

            # Shortcut path
            X_shortcut = Conv2D(256, (1, 1), padding='same')(X_shortcut)
            X_shortcut = BatchNormalization()(X_shortcut)

            X = Add()([X, X_shortcut])
            X = Activation('relu')(X)

            X = Flatten()(X)  # this converts our 3D feature maps to 1D feature vectors
            X = Dense(256)(X)
            X = BatchNormalization()(X)
            X = Dense((env.grid_size**2), activation='linear')(X)   # Output

            model = Model(inputs=X_input, outputs=X, name=MODEL_NAME)
            model.compile(loss="mse", optimizer = "adam", metrics=['accuracy'])
            
        return model

    # Adds step's data to a memory replay array (observation, action, reward, new_observation, done)
    def update_replay_memory(self, transition):
        self.replay_memory.append(transition)
    
    def train(self, terminal_state, step):
        # Start training only if certain number of samples is already saved
        if len(self.replay_memory) < MIN_REPLAY_MEMORY_SIZE:
            return

        # Get a minibatch of random samples from memory replay table
        minibatch = random.sample(self.replay_memory, MINIBATCH_SIZE)
        # Get current states from minibatch, then query NN model for Q values
        current_states = np.array([transition[0] for transition in minibatch])
        current_qs_list = self.model.predict(current_states).reshape(-1, env.grid_size, env.grid_size)
        # Get future states from minibatch, then query NN model for Q values
        # When using target network, query it, otherwise main network should be queried
        new_current_states = np.array([transition[3] for transition in minibatch])
        future_qs_list = self.target_model.predict(new_current_states).reshape(-1, env.grid_size, env.grid_size)
        X = []
        y = []

        # Now we need to enumerate our batches
        for index, (current_state, action, reward, new_current_state, done) in enumerate(minibatch):
            # If not a terminal state, get new q from future states, otherwise set it to 0
            # almost like with Q Learning, but we use just part of equation here
            if not done:
                max_future_q = np.max(future_qs_list[index])
                new_q = reward + DISCOUNT * max_future_q
            else:
                new_q = reward
            # Update Q value for given state
            current_qs = current_qs_list[index]
            current_qs[action] = new_q
            # And append to our training data
            X.append(current_state)
            y.append(current_qs.flatten())

        # Fit on all samples as one batch, log only on terminal state
        self.model.fit(np.array(X), np.array(y), batch_size=MINIBATCH_SIZE, verbose=0, shuffle=False)
                    #    , callbacks=[self.tensorboard] if terminal_state else None)
        # Update target network counter every episode
        if terminal_state:
            self.target_update_counter += 1
        # If counter reaches set value, update target network with weights of main network
        if self.target_update_counter > UPDATE_TARGET_EVERY:
            self.target_model.set_weights(self.model.get_weights())
            self.target_update_counter = 0

    def get_qs(self, state):
        #### DEBUG ####
        # return np.random.uniform(size=(env.grid_size, env.grid_size))
        ###############
        return self.model.predict(state.reshape(-1, *state.shape))[0].reshape(env.grid_size, env.grid_size)

agent = Agent()

## Training

In [ ]:
# For repetitive results
np.random.seed(1234)
random.seed(1234)

# Create models folder
if not os.path.isdir('models'):
    os.makedirs('models')

# For statistics
reward_stats = []

# Iterate over episodes
for episode in tqdm(range(1, EPISODES + 1), unit='episodes', desc='Training'):
    # Update tensorboard step every episode
    # agent.tensorboard.step = episode
    # Reset episode parameters
    episode_reward = 0
    step = 1
    done = False
    # Reset environment + get initial state
    env = VRP()
    current_state = env.get_observation()

    # Iterate until episode ends
    while not done:
        # This part stays mostly the same, the change is to query a model for Q values
        if np.random.random() > EPSILON:
            # Get action from Q table
            action = env.max_valid_action(agent.get_qs(current_state))
        else:
            # Get random action
            action = env.max_valid_action(np.random.uniform(size=(env.grid_size, env.grid_size)))

        new_state, reward, done = env.step(action)
        episode_reward += reward
        # Every step we update replay memory and train main network
        agent.update_replay_memory((current_state, action, reward, new_state, done))
        agent.train(done, step)
        # Update episode parameters
        current_state = new_state
        step += 1

    # Append episode reward to a list and log stats (every given number of episodes)
    reward_stats.append(episode_reward)
    if not episode % AGGREGATE_STATS_EVERY or episode == 1:
        average_reward = sum(reward_stats[-AGGREGATE_STATS_EVERY:])/len(reward_stats[-AGGREGATE_STATS_EVERY:])
        min_reward = min(reward_stats[-AGGREGATE_STATS_EVERY:])
        max_reward = max(reward_stats[-AGGREGATE_STATS_EVERY:])
        # agent.tensorboard.update_stats(reward_avg=average_reward, reward_min=min_reward, reward_max=max_reward, epsilon=epsilon)

        # Save model, but only when min reward is greater or equal a set value
        if min_reward >= MIN_REWARD:
            agent.model.save(f'models/{MODEL_NAME}__{round(average_reward,1)}avg__{int(time.time())}.model')
        print(f"[INFO] episode: {episode} -- AVG reward: {average_reward}")


    # Decay epsilon
    if EPSILON > MIN_EPSILON:
        EPSILON *= EPSILON_DECAY
        EPSILON = max(MIN_EPSILON, EPSILON)

# Save final version
agent.model.save(f'models/{MODEL_NAME}__{round(average_reward,1)}avg__{int(time.time())}.model')

[INFO] episode: 1 -- AVG reward: -156.0
[INFO] episode: 50 -- AVG reward: -144.77800000000005
[INFO] episode: 100 -- AVG reward: -151.32600000000002
[INFO] episode: 150 -- AVG reward: -144.54399999999998
[INFO] episode: 200 -- AVG reward: -148.114
[INFO] episode: 250 -- AVG reward: -145.75400000000002
[INFO] episode: 300 -- AVG reward: -151.43
[INFO] episode: 350 -- AVG reward: -145.62800000000001
[INFO] episode: 400 -- AVG reward: -146.79799999999994
[INFO] episode: 450 -- AVG reward: -146.91000000000005
[INFO] episode: 500 -- AVG reward: -146.85199999999998
[INFO] episode: 550 -- AVG reward: -150.362
[INFO] episode: 600 -- AVG reward: -151.12800000000004
[INFO] episode: 650 -- AVG reward: -148.87199999999996
[INFO] episode: 700 -- AVG reward: -148.91200000000003
[INFO] episode: 750 -- AVG reward: -144.54399999999998
[INFO] episode: 800 -- AVG reward: -144.78400000000002
[INFO] episode: 850 -- AVG reward: -145.404
[INFO] episode: 900 -- AVG reward: -142.87400000000002
[INFO] episode: 

## Visualization

In [ ]:
# %tensorboard --logdir logs

## History

| date | version | performance | comment |
| :--- | :--- | :--- | :--- |
| 07/01 | 2x256C | -75 (10x10 15PoD) | first trainable model but had a bug (not converging after 30h) |
| 13/01 | 2x256C fixed | -75 (10x10 15PoD) | not converging but only 2K episodes |
| 13/01 | 3x256C + 1x128 | -75 (10x10 15PoD) | not converging |
| 14/01 | 3x256C + 1x356 + res | -75 (10x10 15PoD) | ??? |

## Sandbox

In [ ]:
# for i in range(1):
#     # Reset environment
#     env = VRP(grid_size=10, n_pod=10)
#     done = False

#     # for i in range(100):
#     while done == False:
#         action = env.max_valid_action(agent.get_qs(env.get_observation()))
#         # print(action)
#         observation, reward, done = env.step(action)
#         print(observation[:,:,2], '\n')
#         if reward != 0:
#             # print(observation)
#             print(reward)